In [1]:
# dependency imports
from langchain_google_vertexai import VertexAIEmbeddings

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import MarkdownTextSplitter

from langchain_core.documents import Document

import pandas as pd

In [2]:
# Set this to true this if you want to use cloudsql
# USE_CLOUDSQL = False
USE_CLOUDSQL = True

project_id = "imrenagi-gemini-experiment" #change this to your project id
region = "us-central1"
gemini_embedding_model = "text-embedding-004"

if not USE_CLOUDSQL:
    # use pgvector docker image for local development
    database_password = "pyconapac"
    database_name = "pyconapac"
    database_user = "pyconapac"
    database_host = "localhost"
else:
    # use cloudsql credential if you want to use cloudsql
    instance_name="pyconapac-demo"
    database_password = 'testing'
    database_name = 'testing'
    database_user = 'testing'

assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"


In [ ]:
#@markdown ###Authenticate your Google Cloud Account and enable APIs.
# Authenticate gcloud.
# from google.colab import auth
# auth.authenticate_user()

# Configure gcloud.
!gcloud config set project {project_id}

# Grant Cloud SQL Client role to authenticated user
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"
print(f"{current_user}")
# enable aiplatform apiservices

In [ ]:
if USE_CLOUDSQL:
  print(f"Granting Cloud SQL Client role to {current_user[0]}")
  # granting cloudsql client role to the current user
  !gcloud projects add-iam-policy-binding {project_id} \
    --member=user:{current_user[0]} \
    --role="roles/cloudsql.client"
  # Enable Cloud SQL Admin API
  !gcloud services enable sqladmin.googleapis.com

In [ ]:
if USE_CLOUDSQL:
  #@markdown Create and setup a Cloud SQL PostgreSQL instance, if not done already.
  database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
  if database_version[0].startswith("POSTGRES"):
    print("Found an existing Postgres Cloud SQL Instance!")
  else:
    print("Creating new Cloud SQL instance...")
    !gcloud sql instances create {instance_name} --database-version=POSTGRES_15 \
      --region={region} --cpu=1 --memory=4GB --root-password={database_password} \
      --authorized-networks=0.0.0.0/0
  # Create the database, if it does not exist.
  out = !gcloud sql databases list --instance={instance_name} --filter="NAME:{database_name}" --format="value(NAME)"
  if ''.join(out) == database_name:
    print("Database %s already exists, skipping creation." % database_name)
  else:
    !gcloud sql databases create {database_name} --instance={instance_name}
  # Create the database user for accessing the database.
  !gcloud sql users create {database_user} \
    --instance={instance_name} \
    --password={database_password}

In [ ]:
if USE_CLOUDSQL:
    # get the ip address of the instance
    ip_addresses = !gcloud sql instances describe {instance_name} --project {project_id} --format 'value(ipAddresses.ipAddress)'
    # Split the IP addresses and take the first one
    database_host = ip_addresses[0].split(';')[0].strip()
    print(f"Using database host: {database_host}")

In [ ]:
db_conn_string = f"postgres://{database_user}:{database_password}@{database_host}:5432/{database_name}"
db_conn_string

In [ ]:
# Read the JSONL file into a pandas DataFrame
df = pd.read_json('course_content.jsonl', lines=True)
df.head(5)

In [9]:
import asyncpg

async def main():
    # Create connection to PostgreSQL database
    conn = await asyncpg.connect(
        host=database_host,
        user=database_user,
        password=database_password,
        database=database_name
    )

    try:
        await conn.execute("DROP TABLE IF EXISTS course_contents CASCADE")
        # Create the `course_contents` table.
        await conn.execute(
            """CREATE TABLE IF NOT EXISTS course_contents (
                                id SERIAL PRIMARY KEY,
                                title TEXT,
                                content TEXT,
                                file_path TEXT,
                                slug TEXT
                                )"""
        )

        # Create an index on the slug column for faster lookups
        await conn.execute(
            """CREATE INDEX IF NOT EXISTS idx_course_contents_slug 
               ON course_contents (slug)"""
        )

        # Copy the dataframe to the `course_contents` table.
        tuples = list(df.itertuples(index=False))
        await conn.copy_records_to_table(
            "course_contents", records=tuples, columns=list(df), timeout=10
        )
    finally:
        await conn.close()

# Run the SQL commands now.
await main()  # type: ignore

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter

text_splitter = MarkdownTextSplitter(
  chunk_size=1000, 
  chunk_overlap=200)

chunked = []
for index, row in df.iterrows():
    course_content_id = row["id"]
    title = row["title"]
    content = row["content"]
    splits = text_splitter.create_documents([content])
    for s in splits:
        r = {"course_content_id": course_content_id, "content": s.page_content}
        chunked.append(r)

chunked_df = pd.DataFrame(chunked)
chunked_df.head(5)

In [11]:
from langchain_google_vertexai import VertexAIEmbeddings
import time
import vertexai

# Initialize Vertex AI
vertexai.init(project=project_id, location=region)
# Create a Vertex AI Embeddings service
embeddings_service = VertexAIEmbeddings(model_name=gemini_embedding_model)

In [12]:
# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=5, backoff_factor=2, **kwargs):
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)


batch_size = 5
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]]
    response = retry_with_backoff(embeddings_service.embed_documents, request)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response):
        x["embedding"] = e

In [ ]:
# Store the generated embeddings in a pandas dataframe.
course_content_embeddings = pd.DataFrame(chunked)
course_content_embeddings.head()

In [14]:
# Store the generated vector embeddings in a PostgreSQL table.
# This code may run for a few minutes.
import numpy as np
import asyncpg
from pgvector.asyncpg import register_vector

async def main():
    conn = await asyncpg.connect(
        host=database_host,
        user=database_user,
        password=database_password,
        database=database_name
    )

    # this is not used since we already use pgvector docker container
    await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
    await register_vector(conn)

    await conn.execute("DROP TABLE IF EXISTS course_content_embeddings")
    # Create the `product_embeddings` table to store vector embeddings.
    await conn.execute(
        """CREATE TABLE IF NOT EXISTS course_content_embeddings(
                            id INTEGER NOT NULL REFERENCES course_contents(id),
                            content TEXT,
                            embedding vector(768))"""
    )

    # Store all the generated embeddings back into the database.
    for index, row in course_content_embeddings.iterrows():
        await conn.execute(
            "INSERT INTO course_content_embeddings (id, content, embedding) VALUES ($1, $2, $3)",
            row["course_content_id"],
            row["content"],
            np.array(row["embedding"]),
        )

    await conn.close()

# Run the SQL commands now.
await main()  # type: ignore

### Create indexes for faster similarity search in pgvector

- Vector indexes can significantly speed up similarity search operation and avoid the brute-force exact nearest neighbor search that is used by default.

- pgvector comes with two types of indexes (as of v0.5.1): `hnsw` and `ivfflat`.

> 💡 Click [here](https://cloud.google.com/blog/products/databases/faster-similarity-search-performance-with-pgvector-indexes) to learn more about pgvector indexes.

Enter or modify the values of index parameters for your index of choice and run the corresponding cell:

In [15]:
# @markdown Create an HNSW index on the `course_content_embeddings` table:
m =  24 # @param {type:"integer"}
ef_construction = 100  # @param {type:"integer"}
operator =  "vector_cosine_ops"  # @param ["vector_cosine_ops", "vector_l2_ops", "vector_ip_ops"]

# Quick input validations.
assert m, "⚠️ Please input a valid value for m."
assert ef_construction, "⚠️ Please input a valid value for ef_construction."
assert operator, "⚠️ Please input a valid value for operator."

import asyncpg
from pgvector.asyncpg import register_vector

async def main():
    conn = await asyncpg.connect(
        host=database_host,
        user=database_user,
        password=database_password,
        database=database_name
    )
    await register_vector(conn)

    # Create an HNSW index on the `course_content_embeddings` table.
    await conn.execute(
        f"""CREATE INDEX ON course_content_embeddings
          USING hnsw(embedding {operator})
          WITH (m = {m}, ef_construction = {ef_construction})
        """
    )
    await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

In [16]:
query = "what is the best way to design forgot password"  # @param {type:"string"}

assert query, "⚠️ Please input a valid input search text"

qe = embeddings_service.embed_query(query)

In [ ]:
# Convert the query embedding to a numpy array to inspect the content
np.array(qe)

In [ ]:
from pgvector.asyncpg import register_vector
import asyncpg

matches = []

async def main():
    conn = await asyncpg.connect(
        host=database_host,
        user=database_user,
        password=database_password,
        database=database_name
    )
    await register_vector(conn)
    
    similarity_threshold = 0.1
    num_matches = 50

    results = await conn.fetch(
        """
                        WITH vector_matches AS (
                          SELECT id, content, 1 - (embedding <=> $1) AS similarity
                          FROM course_content_embeddings
                          WHERE 1 - (embedding <=> $1) > $2
                          ORDER BY similarity DESC
                          LIMIT $3
                        )
                        SELECT cc.id as id, cc.title as title, 
                            vm.content as content, 
                            vm.similarity as similarity 
                        FROM course_contents cc
                        LEFT JOIN vector_matches vm ON cc.id = vm.id;
                        """,
        qe,
        similarity_threshold,
        num_matches,
        
    )

    if len(results) == 0:
        raise Exception("Did not find any results. Adjust the query parameters.")

    for r in results:
        # Collect the description for all the matched similar contents.
        matches.append(
            {
                "id": r["id"],
                "title": r["title"],
                "content": r["content"],
                "similarity": r["similarity"],                
            }
        )

    await conn.close()


# Run the SQL commands now.
await main()  # type: ignore

matches = pd.DataFrame(matches)
matches.head(10)

# Cleanup

In [19]:
# if USE_CLOUDSQL:
#     !gcloud sql instances delete {instance_name} --quiet